In [43]:
import os
from os import environ

import pickle
from pyquaternion import Quaternion
import numpy as np

import psycopg2
import psycopg2.sql as sql
import postgis.psycopg
import mobilitydb.psycopg


from utils.data_dirs import data_dirs
from utils.index import index
from utils.unique import unique

In [44]:
EXPERIMENT = False

In [45]:
base_dir, output_dir, folder, EXPERIMENT_DATA, suffix, *_ = data_dirs(EXPERIMENT)
print(base_dir)
print(output_dir)
print(folder)
print(EXPERIMENT_DATA)
print(suffix)

/work/apperception/data/raw/nuScenes/full-dataset-v1.0/Mini
/data/apperception-data/processed/nuscenes/full-dataset-v1.0/Mini
v1.0-mini
/work/apperception/data/raw/scenic/experiment_data



In [46]:
connection = psycopg2.connect(
        dbname=environ.get("AP_DB", "mobilitydb"),
        user=environ.get("AP_USER", "docker"),
        host=environ.get("AP_HOST", "localhost"),
        port=environ.get("AP_PORT", "25432"),
        password=environ.get("AP_PASSWORD", "docker"),
    )
connection.autocommit = True
postgis.psycopg.register(connection)
mobilitydb.psycopg.register(connection)

In [47]:
cursor = connection.cursor()

In [48]:
with open(os.path.join(output_dir, 'partition/boston-seaport/ground_truth_annotation.pickle'), 'rb') as f:
    annotations = pickle.load(f)

In [49]:
len(annotations)

7058

In [50]:
annotations = [a for a in annotations if a['category'].startswith('vehicle')]

In [51]:
annotations[0]

{'sample_token': 'de7593d76648450e947ba0c203dee1b0',
 'instance_token': '8ce4fe54af77467d90c840465f69677f',
 'translation': [674.622, 1564.196, 1.623],
 'size': [1.87, 4.837, 2.0],
 'rotation': [0.3243632143817344, 0.0, 0.0, 0.9459326113185595],
 'category': 'vehicle.car',
 'timestamp': 1533151609947025,
 'name': 'scene-0103',
 'location': 'boston-seaport'}

In [52]:
trajectories = {}
for a in annotations:
    oid = a['instance_token']
    if oid not in trajectories:
        trajectories[oid] = []
    trajectories[oid].append(a)

In [53]:
[*trajectories.keys()][:10]

['8ce4fe54af77467d90c840465f69677f',
 'f4af7fd215ee47aa8b64bac0443d7be8',
 'f2d9e6f194ee446eafc7a79fcb32994a',
 'afff27c4b3934c59837d2294137735d5',
 '58ead1aad9b544b4b8126d8723f2764f',
 'c560085506544941af6f891d8a23a6e1',
 '9e5f09bc19cd473dae219194ada64f3d',
 'f5c0c1968ef0486cb3183e521841352f',
 'e561b60694dc4514b6dddcfc5a5b6a90',
 '4bee849ae27b40a684321dbc6f780872']

In [54]:
# direction vector = (1, 0) -> rotate
# then compare the the direction of the segment.

In [55]:
for dets in trajectories.values():
    for det in dets:
        det['direction'] = Quaternion(det['rotation']).rotate([1, 0, 0])

In [56]:
tokens = [*range(len(annotations))]

In [57]:
clss = [*map(lambda x: x['category'], annotations)]

In [58]:
txs = [*map(lambda x: x['translation'][0], annotations)]
tys = [*map(lambda x: x['translation'][1], annotations)]
tzs = [*map(lambda x: x['translation'][2], annotations)]

In [59]:
dxs = [*map(lambda x: x['direction'][0], annotations)]
dys = [*map(lambda x: x['direction'][1], annotations)]

In [60]:
def execute(query, val=None):
    cursor.execute(query, val)
    return cursor.fetchall()

In [61]:
point = sql.SQL("UNNEST({fields}) AS point (token, tx, ty, tz, dx, dy)").format(
    fields=sql.SQL(',').join(map(sql.Literal, [tokens, txs, tys, tzs, dxs, dys]))
)

cursor.execute(sql.SQL("""
SELECT *
FROM {point}
""").format(point=point))
len(cursor.fetchall())

4362

In [62]:
points_in_polygon = sql.SQL("""
SELECT segmenttypes

FROM SegmentPolygon as Polygon
GROUP BY segmenttypes
""").format(point=point)
execute(points_in_polygon)

[(['intersection', 'road'],),
 (['lanegroup'],),
 (['road', 'roadsection'],),
 (['lane', 'lanesection'],)]

In [63]:
points_in_polygon = sql.SQL("""
SELECT token
FROM {point}
JOIN SegmentPolygon as Polygon
    ON ST_Contains(Polygon.elementPolygon, ST_Point(point.tx, point.ty))
    AND ARRAY ['intersection', 'lane', 'lanegroup', 'lanesection'] && Polygon.segmenttypes
GROUP BY token
""").format(point=point)
len(execute(points_in_polygon))

2413

In [64]:
min_polygon = sql.SQL("""
SELECT token, MIN(ST_Area(Polygon.elementPolygon)) as size
FROM {point}
JOIN SegmentPolygon as Polygon
    ON ST_Contains(Polygon.elementPolygon, ST_Point(point.tx, point.ty))
    AND ARRAY ['intersection', 'lane', 'lanegroup', 'lanesection'] && Polygon.segmenttypes
GROUP BY token
""").format(point=point)
len(execute(min_polygon))

2413

In [65]:
len(execute(sql.SQL("""
WITH
MinPolygon AS ({min_polygon})
SELECT token, MIN(elementId) as elementId
FROM {point}
JOIN MinPolygon USING (token)
JOIN SegmentPolygon as Polygon
    ON ST_Contains(Polygon.elementPolygon, ST_Point(point.tx, point.ty))
    AND ST_Area(Polygon.elementPolygon) = MinPolygon.size
GROUP BY token
""").format(point=point, min_polygon=min_polygon)))

2413

In [66]:
function_angle = sql.SQL("""
DROP FUNCTION IF EXISTS angle(double precision);
CREATE OR REPLACE FUNCTION angle(a double precision) RETURNS double precision AS
$BODY$
BEGIN
    RETURN ((a::decimal % 360) + 360) % 360;
END
$BODY$
LANGUAGE 'plpgsql';
""")

In [67]:
_segment_with_direction = sql.SQL("""
SELECT
    *,
    ST_X(endPoint) - ST_X(startPoint) AS _x,
    ST_Y(endPoint) - ST_Y(startPoint) AS _y
FROM Segment
""")
len(execute(_segment_with_direction))

11379

In [68]:
segment_with_direction = sql.SQL("""
SELECT
    *,
    (_x / SQRT(POWER(_x, 2) + POWER(_y, 2))) AS dx,
    (_y / SQRT(POWER(_x, 2) + POWER(_y, 2))) AS dy
FROM ({_segment_with_direction}) AS foo
WHERE
    _x <> 0 OR _y <> 0
""").format(_segment_with_direction=_segment_with_direction)
len(execute(segment_with_direction))

11370

In [69]:
execute(sql.SQL("""
WITH SegmentWithDirection AS ({segment_with_direction})
SELECT *
FROM SegmentWithDirection
WHERE elementId = '8e3c85d1-6d99-44b0-b34a-28a3895485f1'
""").format(segment_with_direction=segment_with_direction))

[(2259,
  '8e3c85d1-6d99-44b0-b34a-28a3895485f1',
  <Point POINT(1016.4165986651158 224.24087740407433)>,
  <Point POINT(1013.7473739613728 221.2467572532935)>,
  <LineString LINESTRING((1016.4165986651158 224.24087740407433), (1013.7473739613728 221.2467572532935))>,
  2.4134998,
  -2.669224703742998,
  -2.994120150780816,
  -0.6654472668690694,
  -0.7464448640164159)]

In [70]:
execute(sql.SQL("""
WITH SegmentWithDirection AS ({segment_with_direction})
SELECT *, ST_X(startPoint), ST_Y(startPoint), ST_X(endPoint), ST_Y(endPoint)
FROM Segment
WHERE elementId = '8e3c85d1-6d99-44b0-b34a-28a3895485f1'
""").format(segment_with_direction=segment_with_direction))

[(2259,
  '8e3c85d1-6d99-44b0-b34a-28a3895485f1',
  <Point POINT(1016.4165986651158 224.24087740407433)>,
  <Point POINT(1013.7473739613728 221.2467572532935)>,
  <LineString LINESTRING((1016.4165986651158 224.24087740407433), (1013.7473739613728 221.2467572532935))>,
  2.4134998,
  1016.4165986651158,
  224.24087740407433,
  1013.7473739613728,
  221.2467572532935)]

In [71]:
polygon_segment = sql.SQL("""
SELECT elementId
FROM SegmentPolygon
""").format(segment_with_direction=segment_with_direction)
len(execute(polygon_segment))

3067

In [72]:

execute(sql.SQL("""
WITH SegmentWithDirection AS ({segment_with_direction})
SELECT elementId, segmenttypes
FROM SegmentPolygon
WHERE NOT EXISTS (
    SELECT *
    FROM SegmentWithDirection
    WHERE SegmentPolygon.elementId = SegmentWithDirection.elementId
)
""").format(segment_with_direction=segment_with_direction))

[]

In [73]:
polygon_segment = sql.SQL("""
WITH SegmentWithDirection AS ({segment_with_direction})
SELECT elementId
FROM SegmentPolygon
JOIN SegmentWithDirection USING (elementId)
GROUP BY elementId
""").format(segment_with_direction=segment_with_direction)
len(execute(polygon_segment))

3067

In [ ]:
min_polygon_id = sql.SQL("""
WITH MinPolygon AS ({min_polygon})
SELECT token, MIN(elementId) as elementId
FROM {point}
JOIN MinPolygon USING (token)
JOIN SegmentPolygon as Polygon
    ON ST_Contains(Polygon.elementPolygon, ST_Point(point.tx, point.ty))
    AND ST_Area(Polygon.elementPolygon) = MinPolygon.size
    AND ARRAY ['intersection', 'lane', 'lanegroup', 'lanesection'] && Polygon.segmenttypes
GROUP BY token
""").format(point=point, min_polygon=min_polygon)
len(execute(min_polygon_id))

2413

In [75]:
execute(
    sql.SQL("""
    WITH MinPolygonId as ({min_polygon_id})

    SELECT segmenttypes
    FROM SegmentPolygon
    JOIN MinPolygonId USING (elementId)
    GROUP BY segmenttypes
    """).format(min_polygon_id=min_polygon_id)
)

[(['intersection', 'road'],), (['lane', 'lanesection'],), (['lanegroup'],)]

[(['intersection', 'road'],), (['lane', 'lanesection'],), (['lanegroup'],)]

In [76]:
point_polygon_segment = sql.SQL("""
WITH
SegmentWithDirection AS ({segment_with_direction}),
MinPolygon AS ({min_polygon}),
MinPolygonId AS ({min_polygon_id})

SELECT token
FROM {point}
JOIN MinPolygonId USING (token)
JOIN SegmentPolygon USING (elementId)
JOIN SegmentWithDirection USING (elementId)
WHERE
    angle(ACOS((point.dx * SegmentWithDirection.dx) + (point.dy * SegmentWithDirection.dy)) * 180 / PI()) < 90
    OR
    angle(ACOS((point.dx * SegmentWithDirection.dx) + (point.dy * SegmentWithDirection.dy)) * 180 / PI()) > 270
    OR
    'intersection' = Any(SegmentPolygon.segmenttypes)
GROUP BY token
    
""").format(
    point=point,
    segment_with_direction=segment_with_direction,
    min_polygon=min_polygon,
    min_polygon_id=min_polygon_id
)
len(execute(point_polygon_segment))

2413

2413

In [ ]:
execute(sql.SQL("""
WITH
SegmentWithDirection AS ({segment_with_direction}),
MinPolygon AS ({min_polygon}),
MinPolygonId AS ({min_polygon_id})

SELECT token, tx, ty, tz, dx, dy, elementId, elementPolygon, segmenttypes
FROM {point}
JOIN MinPolygonId USING (token)
JOIN SegmentPolygon USING (elementId)
WHERE
    NOT EXISTS (
        SELECT *
        FROM SegmentWithDirection
        WHERE SegmentWithDirection.elementId = SegmentPolygon.elementId AND
        (
            angle(ACOS((point.dx * SegmentWithDirection.dx) + (point.dy * SegmentWithDirection.dy)) * 180 / PI()) < 90
            OR
            angle(ACOS((point.dx * SegmentWithDirection.dx) + (point.dy * SegmentWithDirection.dy)) * 180 / PI()) > 270
            OR
            'intersection' = Any(SegmentPolygon.segmenttypes)
        )
    )
""").format(
    point=point,
    segment_with_direction=segment_with_direction,
    min_polygon=min_polygon,
    min_polygon_id=min_polygon_id
))

[]

In [ ]:
segment_map = execute(sql.SQL("""
{function_angle}

WITH
SegmentWithDirection AS ({segment_with_direction}),
MinPolygon AS ({min_polygon}),
MinPolygonId AS ({min_polygon_id}),
PointPolygonSegment AS (
    SELECT
        *,
        ST_Distance(ST_Point(tx, ty), ST_MakeLine(startPoint, endPoint)) AS distance,
        angle(ACOS((point.dx * sd.dx) + (point.dy * sd.dy)) * 180 / PI()) AS anglediff
    FROM {point}
    JOIN MinPolygonId USING (token)
    JOIN SegmentPolygon USING (elementId)
    JOIN SegmentWithDirection AS sd USING (elementId)
    WHERE
        angle(ACOS((point.dx * sd.dx) + (point.dy * sd.dy)) * 180 / PI()) < 90
        OR
        angle(ACOS((point.dx * sd.dx) + (point.dy * sd.dy)) * 180 / PI()) > 270
        OR
        'intersection' = Any(SegmentPolygon.segmenttypes)
),
MinDis as (
    SELECT token, MIN(distance) as mindistance
    FROM PointPolygonSegment
    GROUP BY token
),
MinDisMinAngle as (
    SELECT token, MIN(LEAST(pps.anglediff, 360-pps.anglediff)) as minangle
    FROM PointPolygonSegment AS pps
    JOIN MinDis USING (token)
    WHERE pps.distance = MinDis.mindistance
    GROUP BY token
)

SELECT *
FROM PointPolygonSegment
JOIN MinDis USING (token)
JOIN MinDisMinAngle USING (token)
WHERE PointPolygonSegment.distance = MinDis.mindistance
    AND PointPolygonSegment.anglediff = MinDisMinAngle.minangle


""").format(
    point=point,
    segment_with_direction=segment_with_direction,
    min_polygon=min_polygon,
    min_polygon_id=min_polygon_id,
    function_angle=function_angle
))
cursor.description

(Column(name='token', type_code=23),
 Column(name='elementid', type_code=25),
 Column(name='tx', type_code=1700),
 Column(name='ty', type_code=1700),
 Column(name='tz', type_code=1700),
 Column(name='dx', type_code=1700),
 Column(name='dy', type_code=1700),
 Column(name='elementpolygon', type_code=16391),
 Column(name='segmenttypes', type_code=1009),
 Column(name='segmentid', type_code=23),
 Column(name='startpoint', type_code=16391),
 Column(name='endpoint', type_code=16391),
 Column(name='segmentline', type_code=16391),
 Column(name='heading', type_code=700),
 Column(name='_x', type_code=701),
 Column(name='_y', type_code=701),
 Column(name='dx', type_code=701),
 Column(name='dy', type_code=701),
 Column(name='distance', type_code=701),
 Column(name='anglediff', type_code=701),
 Column(name='mindistance', type_code=701),
 Column(name='minangle', type_code=701))

In [ ]:
[
    {
        **annotations[s[0]],
        'segmentid': s[1],
        'polygonid': s[9]
    }
    for s
    in segment_map
]

[{'sample_token': '0af0feb5b1394b928dd13d648de898f5',
  'instance_token': 'f4af7fd215ee47aa8b64bac0443d7be8',
  'translation': [669.62, 1630.16, 0.547],
  'size': [2.138, 4.918, 1.815],
  'rotation': [-0.49893401461612963, 0.0, 0.0, 0.8666399766102598],
  'category': 'vehicle.car',
  'timestamp': 1533151615448397,
  'name': 'scene-0103',
  'location': 'boston-seaport',
  'direction': [-0.5021296981180634, -0.8647923255139712, 0.0],
  'segmentid': 'c04e23b3-4abe-4bff-8ed7-6017334a0422',
  'polygonid': 3599},
 {'sample_token': 'ce94ef7a0522468e81c0e2b3a2f1e12d',
  'instance_token': 'f4af7fd215ee47aa8b64bac0443d7be8',
  'translation': [669.637, 1630.189, 0.397],
  'size': [2.138, 4.918, 1.815],
  'rotation': [-0.49893401461612963, 0.0, 0.0, 0.8666399766102598],
  'category': 'vehicle.car',
  'timestamp': 1533151619446866,
  'name': 'scene-0103',
  'location': 'boston-seaport',
  'direction': [-0.5021296981180634, -0.8647923255139712, 0.0],
  'segmentid': 'c04e23b3-4abe-4bff-8ed7-6017334a0

In [ ]:
segment_trajectory = {}
for s in segment_map:
    annotation = annotations[s[0]]
    segmentid = s[1]
    polygonid = s[9]
    itoken = annotation['instance_token']
    if itoken not in segment_trajectory:
        segment_trajectory[itoken] = []
    segment_trajectory[itoken].append({
        **annotation,
        'segmentid': segmentid,
        'polygonid': polygonid,
    })

In [ ]:
# segment_trajectory

In [ ]:
def map_points_and_directions_to_segment(annotations: "list"):
    tokens = [*range(len(annotations))]
    clss = [*map(lambda x: x['category'], annotations)]
    txs = [*map(lambda x: x['translation'][0], annotations)]
    tys = [*map(lambda x: x['translation'][1], annotations)]
    tzs = [*map(lambda x: x['translation'][2], annotations)]
    dxs = [*map(lambda x: x['direction'][0], annotations)]
    dys = [*map(lambda x: x['direction'][1], annotations)]
    
    _point = sql.SQL("UNNEST({fields}) AS _point (token, tx, ty, tz, dx, dy)").format(
        fields=sql.SQL(',').join(map(sql.Literal, [tokens, txs, tys, tzs, dxs, dys]))
    )
    
    out = sql.SQL("""
    DROP FUNCTION IF EXISTS angle(double precision);
    CREATE OR REPLACE FUNCTION angle(a double precision) RETURNS double precision AS
    $BODY$
    BEGIN
        RETURN ((a::decimal % 360) + 360) % 360;
    END
    $BODY$
    LANGUAGE 'plpgsql';

    WITH
    Point AS (SELECT * FROM {_point}),
    _SegmentWithDirection AS (
        SELECT
            *,
            ST_X(endPoint) - ST_X(startPoint) AS _x,
            ST_Y(endPoint) - ST_Y(startPoint) AS _y
        FROM Segment
    ),
    SegmentWithDirection AS (
        SELECT
            *,
            (_x / SQRT(POWER(_x, 2) + POWER(_y, 2))) AS dx,
            (_y / SQRT(POWER(_x, 2) + POWER(_y, 2))) AS dy
        FROM _SegmentWithDirection
        WHERE
            _x <> 0 OR _y <> 0
    ),
    MinPolygon AS (
        SELECT token, MIN(ST_Area(Polygon.elementPolygon)) as size
        FROM Point AS p
        JOIN SegmentPolygon AS Polygon
            ON ST_Contains(Polygon.elementPolygon, ST_Point(p.tx, p.ty))
            AND ARRAY ['intersection', 'lane', 'lanegroup', 'lanesection'] && Polygon.segmenttypes
        GROUP BY token
    ),
    MinPolygonId AS (
        SELECT token, MIN(elementId) as elementId
        FROM Point AS p
        JOIN MinPolygon USING (token)
        JOIN SegmentPolygon as Polygon
            ON ST_Contains(Polygon.elementPolygon, ST_Point(p.tx, p.ty))
            AND ST_Area(Polygon.elementPolygon) = MinPolygon.size
            AND ARRAY ['intersection', 'lane', 'lanegroup', 'lanesection'] && Polygon.segmenttypes
        GROUP BY token
    ),
    PointPolygonSegment AS (
        SELECT
            *,
            ST_Distance(ST_Point(tx, ty), ST_MakeLine(startPoint, endPoint)) AS distance,
            angle(ACOS((p.dx * sd.dx) + (p.dy * sd.dy)) * 180 / PI()) AS anglediff
        FROM Point AS p
        JOIN MinPolygonId USING (token)
        JOIN SegmentPolygon USING (elementId)
        JOIN SegmentWithDirection AS sd USING (elementId)
        WHERE
            angle(ACOS((p.dx * sd.dx) + (p.dy * sd.dy)) * 180 / PI()) < 90
            OR
            angle(ACOS((p.dx * sd.dx) + (p.dy * sd.dy)) * 180 / PI()) > 270
            OR
            'intersection' = Any(SegmentPolygon.segmenttypes)
    ),
    MinDis as (
        SELECT token, MIN(distance) as mindistance
        FROM PointPolygonSegment
        GROUP BY token
    ),
    MinDisMinAngle as (
        SELECT token, MIN(LEAST(pps.anglediff, 360-pps.anglediff)) as minangle
        FROM PointPolygonSegment AS pps
        JOIN MinDis USING (token)
        WHERE pps.distance = MinDis.mindistance
        GROUP BY token
    )

    SELECT token, elementid, segmentid
    FROM PointPolygonSegment
    JOIN MinDis USING (token)
    JOIN MinDisMinAngle USING (token)
    WHERE PointPolygonSegment.distance = MinDis.mindistance
        AND PointPolygonSegment.anglediff = MinDisMinAngle.minangle
    """).format(_point=_point)
    
    result = execute(out)
    return result

In [83]:
mapped_annotations = map_points_and_directions_to_segment(annotations)

In [84]:
annotations[:10]

[{'sample_token': 'de7593d76648450e947ba0c203dee1b0',
  'instance_token': '8ce4fe54af77467d90c840465f69677f',
  'translation': [674.622, 1564.196, 1.623],
  'size': [1.87, 4.837, 2.0],
  'rotation': [0.3243632143817344, 0.0, 0.0, 0.9459326113185595],
  'category': 'vehicle.car',
  'timestamp': 1533151609947025,
  'name': 'scene-0103',
  'location': 'boston-seaport',
  'direction': [-0.7895770103118981, 0.6136514847915915, 0.0]},
 {'sample_token': '281b92269fd648d4b52d06ac06ca6d65',
  'instance_token': '8ce4fe54af77467d90c840465f69677f',
  'translation': [674.858, 1564.012, 1.423],
  'size': [1.87, 4.837, 2.0],
  'rotation': [0.3243632143817344, 0.0, 0.0, 0.9459326113185595],
  'category': 'vehicle.car',
  'timestamp': 1533151622948227,
  'name': 'scene-0103',
  'location': 'boston-seaport',
  'direction': [-0.7895770103118981, 0.6136514847915915, 0.0]},
 {'sample_token': '1c9a906c40f6417bbe1cea06d6e55501',
  'instance_token': '8ce4fe54af77467d90c840465f69677f',
  'translation': [674.91

[{'sample_token': 'de7593d76648450e947ba0c203dee1b0',
  'instance_token': '8ce4fe54af77467d90c840465f69677f',
  'translation': [674.622, 1564.196, 1.623],
  'size': [1.87, 4.837, 2.0],
  'rotation': [0.3243632143817344, 0.0, 0.0, 0.9459326113185595],
  'category': 'vehicle.car',
  'timestamp': 1533151609947025,
  'name': 'scene-0103',
  'location': 'boston-seaport',
  'direction': [-0.7895770103118981, 0.6136514847915915, 0.0]},
 {'sample_token': '281b92269fd648d4b52d06ac06ca6d65',
  'instance_token': '8ce4fe54af77467d90c840465f69677f',
  'translation': [674.858, 1564.012, 1.423],
  'size': [1.87, 4.837, 2.0],
  'rotation': [0.3243632143817344, 0.0, 0.0, 0.9459326113185595],
  'category': 'vehicle.car',
  'timestamp': 1533151622948227,
  'name': 'scene-0103',
  'location': 'boston-seaport',
  'direction': [-0.7895770103118981, 0.6136514847915915, 0.0]},
 {'sample_token': '1c9a906c40f6417bbe1cea06d6e55501',
  'instance_token': '8ce4fe54af77467d90c840465f69677f',
  'translation': [674.91